In [135]:
import pandas as pd
import os
import ujson, re
import numpy as np

In [11]:
from src.constants import STORE_LOCATION
from operator import itemgetter

In [5]:
train_df = pd.read_csv(os.path.join(STORE_LOCATION, 'train.csv'))

In [6]:
test_df = pd.read_csv(os.path.join(STORE_LOCATION, 'test.csv'))

## Accuracy

In [10]:
train_df.head()

,QuestionId,AcceptedAnswerUserId,AnswerCreationDate,UsersSorted,UsersJaccard
0,5.0,45.0,2010-07-20 19:21:52.240,"[45.0, 1102.0, 48.0, 226.0, 242.0]","{48.0: 0.5714285714285714, 242.0: 0.0135135135..."
1,1.0,8.0,2010-07-20 19:22:20.193,"[8.0, 49.0, 25.0, 35.0, 39.0]","{8.0: 0.042253521126760563, 49.0: 0.5999999999..."
2,8.0,38.0,2010-07-20 19:23:54.230,"[38.0, 173.0, 334.0, 53.0, 55.0]","{55.0: 0.44444444444444442, 334.0: 0.078431372..."
3,2.0,34.0,2010-07-20 19:24:52.200,"[34.0, 56.0, 35.0, 190287.0, 72.0]","{56.0: 0.071428571428571425, 72.0: 0.020689655..."
4,20.0,38.0,2010-07-20 19:32:21.590,"[38.0, 72.0, 232.0, 51.0, 210.0]","{72.0: 0.020689655172413793, 232.0: 0.00612244..."


In [91]:
def get_user_js_vals(jc):
    return re.findall(r'[{]*(\d+).0: (\d+[.]{0,1}\d*)[,}]', jc)

In [100]:
def get_sorted_jc(row):
    user_js = row.UsersJaccard
    user_js_vals = get_user_js_vals(user_js)
    
    user_js_int = map(lambda x: (float(x[0]), float(x[1])), user_js_vals)
    user_js_sorted = sorted(user_js_int, key=itemgetter(1), reverse=True)
    
    return user_js_sorted

In [101]:
def get_best_user_jc(row):
    user_js_sorted = get_sorted_jc(row)
    return user_js_sorted[0][0]

In [157]:
def get_accuracy(df):
    tt = df.apply(get_best_user_jc, axis=1)
    acc = (df.AcceptedAnswerUserId.values == tt).sum() * 1./ df.shape[0]
    
    return acc

In [158]:
print get_accuracy(train_df), get_accuracy(test_df)

0.0358318561661 0.0581283164496


## MRR

In [108]:
def get_ranked_user_jc(row):
    user_js_sorted = get_sorted_jc(row)
    return map(lambda x: x[0], user_js_sorted)

In [155]:
def get_mrr(df):
    tt = df.apply(get_ranked_user_jc, axis=1).values
    ttt = tt - df.AcceptedAnswerUserId.values.reshape(df.AcceptedAnswerUserId.values.shape[0], 1)
    tttt = np.where(ttt==0)[1] + 1

    return np.sum(1./tttt) / df.shape[0]

In [156]:
print get_mrr(train_df), get_mrr(test_df)

0.302688444332 0.34449670365
